In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!pip install -r https://raw.githubusercontent.com/Project-MONAI/MONAI/dev/requirements-dev.txt
!pip install volumentations-3D

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np
import os

import tensorflow as tf
from tensorflow import keras

import torch
from torch import nn
import torch.nn.functional as tnf
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torch.optim as optim

import sklearn
from sklearn.model_selection import KFold



from volumentations import *
import nibabel as nib
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai import transforms
from monai.transforms import (
    AsDiscrete,
    Activations,
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.utils.enums import MetricReduction
from monai.networks.nets import SwinUNETR, UNETR
from monai import data
from monai.data import decollate_batch
from functools import partial


In [ ]:
root_dir = "/content/gdrive/MyDrive/Summer Programs/HECKTOR2022/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task501_Hecktor"
batch_size = 1
num_workers = 2
total_epoch = 200
lr = 0.0001
device = torch.device("cuda:0")

### Model Creation

In [ ]:
model = SwinUNETR(
    img_size=(160, 160, 64),
    in_channels=2,
    out_channels=3,
    feature_size=48,
    drop_rate=0.0,
    attn_drop_rate=0.0,
    dropout_path_rate=0.0
).to(device)

# model = UNETR(
#     img_size=(160, 160, 64),
#     in_channels=2,
#     out_channels=2,
#     feature_size=48
# ).to(device)

### Dataset Creation


In [ ]:
def get_augmentation_train(patch_size):
    return Compose([
        Rotate((-8, 8), (-8, 8), (-8, 8), p=0.5),
        RandomCrop(shape = (160, 160, 64), p = 1.0),
        ElasticTransform((0, 0.25), interpolation=2, p=0.1),
        #GaussianNoise(var_limit=(0, 5), p=0.2),
        #RandomGamma(gamma_limit=(0.5, 1.5), p=0.2),
    ], p=1.0)

In [ ]:
def get_augmentation_test(patch_size):
    return Compose([
        RandomCrop(shape = (160, 160, 64), p = 1.0)
    ], p=1.0)

In [ ]:
def tissue_thresholding(img):
  min, max = -175, 275
  img = np.clip(img, min, max)
  img = (img - min) / (max - min)
  return img
def pet_rescaling(img):
  min, max = np.min(img), np.max(img)
  img = (img - min) / (max - min)
  return img

In [ ]:
def load_data(root_dir, filename, split, aug):
  filename = filename.upper()
  dir_img = os.path.join(root_dir, "imagesTr")
  dir_seg = os.path.join(root_dir, "labelsTr")
  if split == 'test':
    dir_img = os.path.join(root_dir, "imagesTs")
    dir_seg = os.path.join(root_dir, "labelsTs")
  filename_ct = os.path.join(dir_img,  filename[:-7] + '_0000.nii.gz')
  filename_pt = os.path.join(dir_img, filename[:-7] + '_0001.nii.gz')
  filename_mask = os.path.join(dir_seg, filename[:-7] + '.nii.gz')
  ct_img = np.expand_dims(nib.load(filename_ct).get_fdata(), axis = 3)
  pet_img = np.expand_dims(nib.load(filename_pt).get_fdata(), axis = 3)
  mask = nib.load(filename_mask).get_fdata()
  # mask_1 = np.expand_dims((mask==1.).astype(np.float32), axis = 3)
  # mask_2 = np.expand_dims((mask==2.).astype(np.float32), axis = 3)
  # mask = np.concatenate([mask_0, mask_1, mask_2], axis=3)
  mask = keras.utils.to_categorical(
   mask, num_classes=3, dtype='float32'
  )
  img = np.concatenate((ct_img, pet_img), axis = 3)
  if aug: 
    if(split == 'train'):
      aug = get_augmentation_train((160, 160, 64))
    else:
      aug = get_augmentation_test((160, 160, 64))
    data = {'image': img, 'mask': mask}
    aug_data = aug(**data)
    img, mask = aug_data['image'], aug_data['mask']
    img, mask = img.transpose(3 , 0, 1, 2).astype(np.float32), mask.transpose(3, 0, 1, 2).astype(np.float32)
    return img, mask
  else:
    img, mask = img.transpose(3 , 0, 1, 2), mask.transpose(3, 0, 1, 2)
    return img, mask

In [ ]:
class HEKDataset(Dataset):
    def __init__(self, root_dir, split='train'):
        self.root_dir = root_dir
        self.split = split
        self.image_paths = []
        # Load data index
        if split == 'train':
          for path in os.listdir(os.path.join(root_dir, "labelsTr")):
                  self.image_paths.append(path)
        else:
          for path in os.listdir(os.path.join(root_dir, "labelsTs")):
              self.image_paths.append(path)
        print('Succesfully loaded {} dataset.'.format(split) + ' '*50)
            
            
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        filename = self.image_paths[idx]
        return load_data(self.root_dir, filename, self.split, True)

In [ ]:
train_set = HEKDataset(root_dir, split='train')
test_set = HEKDataset(root_dir, split='test')
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True,)
val_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True,)

Succesfully loaded train dataset.                                                  
Succesfully loaded test dataset.                                                  


###Metrics + Losses


In [ ]:
def compute_dice_coef(inputs, targets, smooth = 1e-7):
  #inputs = tnf.softmax(inputs)
  inputs = inputs.view(-1)
  targets = targets.view(-1)
  
  intersection = (inputs * targets).sum()                            
  dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
  return (2.*intersection), (inputs.sum() + targets.sum())  

In [ ]:
def flatten(tensor):
    """Flattens a given tensor such that the channel axis is first.
    The shapes are transformed as follows:
       (N, C, D, H, W) -> (C, N * D * H * W)
    """
    tensor = tensor[:, 1:, :, :, :]
    # number of channels
    C = tensor.size(1)
    # new axis order
    axis_order = (1, 0) + tuple(range(2, tensor.dim()))
    # Transpose: (N, C, D, H, W) -> (C, N, D, H, W)
    transposed = tensor.permute(axis_order)
    # Flatten: (C, N, D, H, W) -> (C, N * D * H * W)
    return transposed.contiguous().view(C, -1)
    
def compute_per_channel_dice(input, target, epsilon=1e-6, weight=None):
    """
    Computes DiceCoefficient as defined in https://arxiv.org/abs/1606.04797 given  a multi channel input and target.
    Assumes the input is a normalized probability, e.g. a result of Sigmoid or Softmax function.
    Args:
         input (torch.Tensor): NxCxSpatial input tensor
         target (torch.Tensor): NxCxSpatial target tensor
         epsilon (float): prevents division by zero
         weight (torch.Tensor): Cx1 tensor of weight per channel/class
    """

    # input and target shapes must match
    assert input.size() == target.size(), "'input' and 'target' must have the same shape"

    input = flatten(input)
    target = flatten(target)
    target = target.float()

    # compute per channel Dice Coefficient
    intersect = (input * target).sum(-1)
    if weight is not None:
        intersect = weight * intersect

    # here we can use standard dice (input + target).sum(-1) or extension (see V-Net) (input^2 + target^2).sum(-1)
    denominator = (input * input).sum(-1) + (target * target).sum(-1)
    return 2 * (intersect / denominator.clamp(min=epsilon))

In [ ]:
class _AbstractDiceLoss(nn.Module):
    """
    Base class for different implementations of Dice loss.
    """

    def __init__(self, weight=None, normalization='sigmoid'):
        super(_AbstractDiceLoss, self).__init__()
        self.register_buffer('weight', weight)
        # The output from the network during training is assumed to be un-normalized probabilities and we would
        # like to normalize the logits. Since Dice (or soft Dice in this case) is usually used for binary data,
        # normalizing the channels with Sigmoid is the default choice even for multi-class segmentation problems.
        # However if one would like to apply Softmax in order to get the proper probability distribution from the
        # output, just specify `normalization=Softmax`
        assert normalization in ['sigmoid', 'softmax', 'none']
        if normalization == 'sigmoid':
            self.normalization = nn.Sigmoid()
        elif normalization == 'softmax':
            self.normalization = nn.Softmax(dim=1)
        else:
            self.normalization = lambda x: x

    def dice(self, input, target, weight):
        # actual Dice score computation; to be implemented by the subclass
        raise NotImplementedError

    def forward(self, input, target):
        # get probabilities from logits
        input = self.normalization(input)

        # compute per channel Dice coefficient
        per_channel_dice = self.dice(input, target, weight=self.weight)

        # average Dice score across all channels/classes
        return 1. - torch.mean(per_channel_dice)


class DiceLoss(_AbstractDiceLoss):
    """Computes Dice Loss according to https://arxiv.org/abs/1606.04797.
    For multi-class segmentation `weight` parameter can be used to assign different weights per class.
    The input to the loss function is assumed to be a logit and will be normalized by the Sigmoid function.
    """

    def __init__(self, weight=None, normalization='sigmoid'):
        super().__init__(weight, normalization)

    def dice(self, input, target, weight):
        return compute_per_channel_dice(input, target, weight=self.weight)

In [ ]:
import torch
import torch.nn as nn


# Helper function to enable loss function to be flexibly used for 
# both 2D or 3D image segmentation - source: https://github.com/frankkramer-lab/MIScnn

def identify_axis(shape):
    # Three dimensional
    if len(shape) == 5 : return [2,3,4]

    
    # Exception - Unknown
    else : raise ValueError('Metric: Shape of tensor is neither 2D or 3D.')


class AsymmetricFocalLoss(nn.Module):
    """For Imbalanced datasets
    Parameters
    ----------
    delta : float, optional
        controls weight given to false positive and false negatives, by default 0.25
    gamma : float, optional
        Focal Tversky loss' focal parameter controls degree of down-weighting of easy examples, by default 2.0
    epsilon : float, optional
        clip values to prevent division by zero error
    """
    def __init__(self, delta=0.7, gamma=2., epsilon=1e-07):
        super(AsymmetricFocalLoss, self).__init__()
        self.delta = delta
        self.gamma = gamma
        self.epsilon = epsilon

    def forward(self, y_pred, y_true):
        y_pred = torch.clamp(y_pred, self.epsilon, 1. - self.epsilon)
        cross_entropy = -y_true * torch.log(y_pred)
        
	# Calculate losses separately for each class, only suppressing background class
        back_ce = torch.pow(1 - y_pred[:,0,:,:], self.gamma) * cross_entropy[:,0,:,:]
        back_ce =  (1 - self.delta) * back_ce

        fore_ce = cross_entropy[:,1:,:,:]
        fore_ce = self.delta * fore_ce
        #print(fore_ce.size())
        loss = torch.mean(torch.sum(torch.stack([back_ce, fore_ce[:,0], fore_ce[:,1]], axis=-1), axis=-1))

        return loss


class AsymmetricFocalTverskyLoss(nn.Module):
    """This is the implementation for binary segmentation.
    Parameters
    ----------
    delta : float, optional
        controls weight given to false positive and false negatives, by default 0.7
    gamma : float, optional
        focal parameter controls degree of down-weighting of easy examples, by default 0.75
    smooth : float, optional
        smooithing constant to prevent division by 0 errors, by default 0.000001
    epsilon : float, optional
        clip values to prevent division by zero error
    """
    def __init__(self, delta=0.7, gamma=0.75, epsilon=1e-07):
        super(AsymmetricFocalTverskyLoss, self).__init__()
        self.delta = delta
        self.gamma = gamma
        self.epsilon = epsilon

    def forward(self, y_pred, y_true):
        # Clip values to prevent division by zero error
        y_pred = torch.clamp(y_pred, self.epsilon, 1. - self.epsilon)
        axis = identify_axis(y_true.size())

        # Calculate true positives (tp), false negatives (fn) and false positives (fp)     
        tp = torch.sum(y_true * y_pred, axis=axis)
        fn = torch.sum(y_true * (1-y_pred), axis=axis)
        fp = torch.sum((1-y_true) * y_pred, axis=axis)
        dice_class = (tp + self.epsilon)/(tp + self.delta*fn + (1-self.delta)*fp + self.epsilon)

        # Calculate losses separately for each class, only enhancing foreground class
        back_dice = (1-dice_class[:,0]) 
        fore_dice = (1-dice_class[:,1:]) * torch.pow(1-dice_class[:,1:], -self.gamma) 
        #print(fore_dice.size())
        # Average class scores
        loss = torch.mean(torch.stack([back_dice,fore_dice[:, 0], fore_dice[:, 1]], axis=-1))
        return loss



class AsymmetricUnifiedFocalLoss(nn.Module):
    """The Unified Focal loss is a new compound loss function that unifies Dice-based and cross entropy-based loss functions into a single framework.
    Parameters
    ----------
    weight : float, optional
        represents lambda parameter and controls weight given to asymmetric Focal Tversky loss and asymmetric Focal loss, by default 0.5
    delta : float, optional
        controls weight given to each class, by default 0.6
    gamma : float, optional
        focal parameter controls the degree of background suppression and foreground enhancement, by default 0.5
    epsilon : float, optional
        clip values to prevent division by zero error
    """
    def __init__(self, weight=0.5, delta=0.6, gamma=0.2):
        super(AsymmetricUnifiedFocalLoss, self).__init__()
        self.weight = weight
        self.delta = delta
        self.gamma = gamma

    def forward(self, y_pred, y_true):
      # Obtain Asymmetric Focal Tversky loss
      activation = nn.Softmax(dim = 1)
      y_pred = activation(y_pred)
      asymmetric_ftl = AsymmetricFocalTverskyLoss(delta=self.delta, gamma=self.gamma)(y_pred, y_true)

      # Obtain Asymmetric Focal loss
      asymmetric_fl = AsymmetricFocalLoss(delta=self.delta, gamma=self.gamma)(y_pred, y_true)

      # Return weighted sum of Asymmetrical Focal loss and Asymmetric Focal Tversky loss
      if self.weight is not None:
        return (self.weight * asymmetric_ftl) + ((1-self.weight) * asymmetric_fl)  
      else:
        return asymmetric_ftl + asymmetric_fl

In [ ]:

# input = torch.randn((1, 2, 160, 160, 64)).to(device)
# output = model(input)
optimizer = optim.Adam(model.parameters(), lr=lr)
#seg_loss = DiceLoss(norma1ization = 'softmax')
seg_loss = AsymmetricUnifiedFocalLoss()
#print([e.shape for e in output])

### Training Loop

In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/Summer Programs/HECKTOR2022/Saved Models/SWIN_UNETR_new.pth'))

<All keys matched successfully>

In [ ]:
num_train = len(train_set)
indices = list(range(num_train))
split = int(np.floor(0.2 * num_train))
np.random.seed(42)
np.random.shuffle(indices)

In [ ]:
train_idx, valid_idx = indices[split:], indices[:split]
train_subsampler = SubsetRandomSampler(train_idx)
val_subsampler = SubsetRandomSampler(valid_idx)
train_loader = DataLoader(
                      train_set, 
                      batch_size=batch_size, 
                      num_workers=num_workers,
                      pin_memory=True, sampler=train_subsampler)
val_loader = DataLoader(
                      train_set,
                      batch_size=batch_size, 
                      num_workers=num_workers,
                      pin_memory=True, sampler=val_subsampler)

In [ ]:
nb_train_batches = len(train_loader)
nb_val_batches = len(val_loader)
nb_iter = 0
best_val_DC = 0.
iters = list(range(1, 10))
val_losses = []
train_losses = []
train_accuracy=[]
val_accuracy=[]
curr_epoch = 57
best_val_DC = 0

In [ ]:
while curr_epoch  < total_epoch:
    train_loss, val_loss = 0., 0.
    train_dsc_1, val_dsc_1 = 0., 0.
    train_dsc_2, val_dsc_2 = 0., 0.
    intersection1, union1 = 0,0
    intersection2, union2 = 0,0
    ############
    # TRAINING #
    ############
    model.train()
    train_data = iter(train_loader)
    for k in range(nb_train_batches):
        imgs, seg_gts = train_data.next()
        #print(imgs.size(), seg_gts.size(), device)
        imgs, seg_gts = imgs.to(device), seg_gts.to(device)

        # Forward pass
        logits = model(imgs)
        #print(logits.size, imgs.size)
        loss = seg_loss(logits, seg_gts)
 
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() / nb_train_batches
        
        train_losses.append(train_loss)
        
        with torch.no_grad():
            preds = torch.argmax(logits, axis = 1)
            pred_1 = (preds==1.).type(torch.float32)
            pred_2 = (preds==2.).type(torch.float32)

            gt_1 = seg_gts[:,1,:,:, :].type(torch.int8)
            gt_2 = seg_gts[:,2,:,:, :].type(torch.int8)
            inter1, u1 = compute_dice_coef(pred_1, gt_1)
            inter2, u2 = compute_dice_coef(pred_2, gt_2)
            intersection1 += inter1
            intersection2 += inter2
            union1 += u1
            union2 += u2

        train_dsc_1 = intersection1 / union1
        train_dsc_2 = intersection2 / union2

            
        # Increase iterations
        nb_iter += 1
        
        print('\rEpoch {}, iter {}/{}, loss {:.6f}'.format(curr_epoch+1, k+1, nb_train_batches, loss.item()), 
              end='')
    print('\rEpoch {}, iter {}/{}, loss {:.6f}'.format(curr_epoch+1, k+1, nb_train_batches, train_loss), 
              end='')
    print()
    intersection1, union1 = 0,0
    intersection2, union2 = 0,0
    ##############
    # VALIDATION #
    ##############
    model.eval()
    with torch.no_grad():
        val_data = iter(val_loader)
        for k in range(nb_val_batches):
            # Loads data
            imgs, seg_gts = val_data.next()
            imgs, seg_gts = imgs.to(device), seg_gts.to(device)
            
            # Forward pass
            logits = model(imgs)
            val_loss += seg_loss(logits, seg_gts).item() / nb_val_batches
            
            val_losses.append(val_loss)

            # Std out
            print('\rValidation iter {}/{}'.format(k+1, nb_val_batches), end='')
            
            # Compute segmentation metric
            preds = torch.argmax(logits, axis = 1)
            pred_1 = (preds==1.).type(torch.float32)
            pred_2 = (preds==2.).type(torch.float32)

            #pred_1 = (logits[:,0,:,:, :]>=0.5).type(torch.int8).cpu().to(device)
      
            #pred_2 = (logits[:,1,:,:, :]>=0.5).type(torch.int8).cpu().to(device)
  
            gt_1 = seg_gts[:,1,:,:, :].type(torch.int8)
            gt_2 = seg_gts[:,2,:,:, :].type(torch.int8)
            dsc_1 = compute_dice_coef(pred_1, gt_1)
            dsc_2 = compute_dice_coef(pred_2, gt_2)

            inter1, u1 = compute_dice_coef(pred_1, gt_1)
            inter2, u2 = compute_dice_coef(pred_2, gt_2)
            intersection1 += inter1
            intersection2 += inter2
            union1 += u1
            union2 += u2

        val_dsc_1 = intersection1 / union1
        val_dsc_2 = intersection2 / union2
    print('\nEpoch {}, Class 1 Train DC: {:.6f}, Class 2 Train DC: {:.6f}, Val Loss:{:.6f}, Class 1 Val DC:{:.6f}, Class 2 Val DC:{:.6f}'.format(curr_epoch + 1, train_dsc_1, train_dsc_2, val_loss, val_dsc_1, val_dsc_2))
    if val_dsc_1 + val_dsc_2 > best_val_DC:
        torch.save(model.state_dict(), '/content/gdrive/MyDrive/Summer Programs/HECKTOR2022/Saved Models/SWIN_UNETR_final.pth')
        best_val_DC = val_dsc_1 + val_dsc_2
        print('Best validation DC reached on epoch. Saved model weights.')
    print('_'*50)
        
    # End of epoch
    curr_epoch  += 1


Epoch 58, iter 453/453, loss 0.115703
Validation iter 70/70
Epoch 58, Class 1 Train DC: 0.836261, Class 2 Train DC: 0.809865, Val Loss:0.182402, Class 1 Val DC:0.802431, Class 2 Val DC:0.662214
Best validation DC reached on epoch. Saved model weights.
__________________________________________________
Epoch 59, iter 453/453, loss 0.113858
Validation iter 70/70
Epoch 59, Class 1 Train DC: 0.841507, Class 2 Train DC: 0.813089, Val Loss:0.179423, Class 1 Val DC:0.803774, Class 2 Val DC:0.664127
Best validation DC reached on epoch. Saved model weights.
__________________________________________________
Epoch 60, iter 453/453, loss 0.112844
Validation iter 70/70
Epoch 60, Class 1 Train DC: 0.842795, Class 2 Train DC: 0.816055, Val Loss:0.178404, Class 1 Val DC:0.805139, Class 2 Val DC:0.637140
__________________________________________________
Epoch 61, iter 453/453, loss 0.113486
Validation iter 70/70
Epoch 61, Class 1 Train DC: 0.842510, Class 2 Train DC: 0.814173, Val Loss:0.183365, Clas

### Data Visualization

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
def show_mri(img, mask):
    for i in range(img.shape[3]):
        fig = plt.figure(figsize=(10, 7))
        fig.add_subplot(1, 2, 1)
        plt.axis(False)
        plt.imshow(img[0, :, :, i], cmap='gray')
        fig.add_subplot(1, 2, 2)
        plt.axis(False)
        plt.imshow(mask[0, :, :, i], cmap='gray')
        clear_output(wait=True)
        plt.show()
def show_mri_slice(img, mask, slice, pred = None):
    fig = plt.figure(figsize=(10, 7))
    if(pred is  None):
      fig.add_subplot(1, 2, 1)
      plt.axis(False)
      plt.imshow(img[0, :, :, slice], cmap='gray')
      fig.add_subplot(1, 2, 2)
      plt.axis(False)
      plt.imshow(mask[0, :, :, slice], cmap='gray')
    else:
      fig.add_subplot(1, 3, 1)
      plt.axis(False)
      plt.imshow(img[0, :, :, slice], cmap='gray')
      fig.add_subplot(1, 3, 2)
      plt.axis(False)
      plt.imshow(mask[0, :, :, slice], cmap='gray')
      fig.add_subplot(1, 3, 3)
      plt.axis(False)
      plt.imshow(pred[0, :, :, slice], cmap='gray')
    plt.show()

In [ ]:
img, mask = load_data(root_dir, "HEK_001.nii.gz", 'train', True)

In [ ]:
pred = np.concatenate([np.expand_dims(pred_1, axis = 0), np.expand_dims(pred_2, axis = 0)], axis = 0)

In [ ]:
print(img.shape, mask.shape, pred.shape)

In [ ]:
show_mri_slice(img, mask, 15, pred)

In [ ]:
show_mri(img, mask)